In [19]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, TokenClassificationPipeline
from model2 import get_model
import torch

model_name = "QCRI/bert-base-multilingual-cased-pos-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = get_model()


Some weights of the model checkpoint at QCRI/bert-base-multilingual-cased-pos-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [20]:
import nltk
from transformers import AutoTokenizer, AutoModelForTokenClassification
from torch.utils.data import Dataset, DataLoader

# Download the Penn TreeBank dataset
nltk.download('treebank')

[nltk_data] Downloading package treebank to
[nltk_data]     /u/amo-d1/grad/mha361/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.


True

In [21]:
from nltk.corpus import treebank

# Load the dataset
sentences = treebank.tagged_sents()


In [26]:
len(sentences)

3914

In [18]:
text = "A test example"
model = model.to("cuda")
inputs = tokenizer(text, return_tensors="pt").to("cuda")
outputs = model(**inputs)
logits = outputs.logits
predictions = torch.argmax(logits, dim=-1)
predictions = predictions.cpu().numpy()

# Convert to labels
label_list = model.config.id2label
labeled_predictions = [label_list[id] for id in predictions[0]]

# Combine tokens and predictions
tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
result = list(zip(tokens, labeled_predictions))

print(result)

[('[CLS]', 'NN'), ('A', 'DT'), ('test', 'NN'), ('example', 'NN'), ('[SEP]', 'NN')]


In [8]:
for n,m in model.named_modules():
    print(n)
    # print(m)


bert
bert.embeddings
bert.embeddings.word_embeddings
bert.embeddings.position_embeddings
bert.embeddings.token_type_embeddings
bert.embeddings.LayerNorm
bert.embeddings.dropout
bert.encoder
bert.encoder.layer
bert.encoder.layer.0
bert.encoder.layer.0.attention
bert.encoder.layer.0.attention.self
bert.encoder.layer.0.attention.self.query
bert.encoder.layer.0.attention.self.key
bert.encoder.layer.0.attention.self.value
bert.encoder.layer.0.attention.self.dropout
bert.encoder.layer.0.attention.output
bert.encoder.layer.0.attention.output.dense
bert.encoder.layer.0.attention.output.LayerNorm
bert.encoder.layer.0.attention.output.dropout
bert.encoder.layer.0.intermediate
bert.encoder.layer.0.intermediate.dense
bert.encoder.layer.0.intermediate.intermediate_act_fn
bert.encoder.layer.0.output
bert.encoder.layer.0.output.dense
bert.encoder.layer.0.output.LayerNorm
bert.encoder.layer.0.output.dropout
bert.encoder.layer.1
bert.encoder.layer.1.attention
bert.encoder.layer.1.attention.self
bert.e

In [32]:
import nltk
import torch
import numpy as np
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForTokenClassification, AdamW
from sklearn.metrics import f1_score

# Download the Penn TreeBank dataset
nltk.download('treebank')

from nltk.corpus import treebank

# Load the dataset
sentences = treebank.tagged_sents()

# Split the dataset into train and test sets
train_size = int(0.8 * len(sentences))
train_sentences = sentences[:train_size]
test_sentences = sentences[train_size:]

class PennTreebankDataset(Dataset):
    def __init__(self, sentences, tokenizer, max_length=128):
        self.sentences = sentences
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.tag2id = {tag: i for i, tag in enumerate(set([tag for sent in self.sentences for _, tag in sent]))}
        self.id2tag = {i: tag for tag, i in self.tag2id.items()}

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        sentence, tags = zip(*self.sentences[idx])
        sentence = " ".join(sentence)
        
        encoding = self.tokenizer(sentence, truncation=True, padding='max_length', max_length=self.max_length, return_tensors='pt')
        
        # Align tags with tokenized words
        aligned_tags = []
        words = self.tokenizer.tokenize(sentence)
        for word, tag in zip(sentence.split(), tags):
            tokens = self.tokenizer.tokenize(word)
            aligned_tags.extend([tag] * len(tokens))
        
        # Pad or truncate tags to match input_ids length
        if len(aligned_tags) > self.max_length - 2:
            aligned_tags = aligned_tags[:self.max_length - 2]
        
        # Add tags for [CLS] and [SEP]
        aligned_tags = [aligned_tags[0]] + aligned_tags + [aligned_tags[-1]]
        
        # Pad with the last tag if necessary
        aligned_tags += [aligned_tags[-1]] * (self.max_length - len(aligned_tags))
        
        # Convert tags to IDs
        tag_ids = [self.tag2id[tag] for tag in aligned_tags]
        
        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.tensor(tag_ids)
        }
# Set up the model and tokenizer
model_name = "QCRI/bert-base-multilingual-cased-pos-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

# Create datasets and dataloaders
train_dataset = PennTreebankDataset(train_sentences, tokenizer)
test_dataset = PennTreebankDataset(test_sentences, tokenizer)

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16)

# Training setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = AdamW(model.parameters(), lr=5e-5)

# Training loop
num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    for batch in tqdm(train_dataloader, desc=f"Training Epoch {epoch+1}"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

# Evaluation and F1 score calculation
model.eval()
all_predictions = []
all_labels = []

with torch.no_grad():
    for batch in tqdm(test_dataloader, desc="Evaluating"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        
        # Remove padding and special tokens
        for pred, label, mask in zip(predictions, labels, attention_mask):
            pred = pred[mask.bool()][1:-1]
            label = label[mask.bool()][1:-1]
            all_predictions.extend(pred.cpu().numpy())
            all_labels.extend(label.cpu().numpy())

# Calculate F1 score
f1 = f1_score(all_labels, all_predictions, average='weighted')
print(f"Weighted F1 Score: {f1:.4f}")

# Print classification report
from sklearn.metrics import classification_report
print("\nClassification Report:")
print(classification_report(all_labels, all_predictions, target_names=list(train_dataset.tag2id.keys())))

[nltk_data] Downloading package treebank to
[nltk_data]     /u/amo-d1/grad/mha361/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
/u/amo-d1/grad/mha361/anaconda3/envs/memit/lib/python3.9/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of the model checkpoint at QCRI/bert-base-multilingual-cased-pos-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPr

Weighted F1 Score: 0.1134

Classification Report:
              precision    recall  f1-score   support

          RP       0.69      0.85      0.76        34
         JJS       0.90      0.98      0.94        45
          RB       0.95      0.98      0.97       630
          TO       1.00      1.00      1.00       464
         JJR       0.92      0.95      0.94        84
          IN       0.99      0.98      0.99      1953
         PDT       0.00      0.00      0.00         6
         SYM       0.00      0.00      0.00        28
         WRB       0.00      0.00      0.00       602
          LS       0.00      0.00      0.00       104
         VBN       0.00      0.00      0.00       597
       -RRB-       0.00      0.00      0.00       388
          VB       0.00      0.00      0.00      1768
         VBZ       0.00      0.00      0.00      3741
          JJ       0.00      0.00      0.00       930
           ,       0.00      0.00      0.00       106
          NN       0.00      0.

/u/amo-d1/grad/mha361/anaconda3/envs/memit/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/u/amo-d1/grad/mha361/anaconda3/envs/memit/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/u/amo-d1/grad/mha361/anaconda3/envs/memit/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg

In [41]:
import nltk
from nltk.corpus import treebank
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch
from transformers import pipeline
from sklearn.metrics import classification_report

# Download the treebank dataset
nltk.download('treebank')

# Load the model and tokenizer
model_name = "QCRI/bert-base-multilingual-cased-pos-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

# Get the tag set and label mapping
label_list = model.config.id2label

# Load the dataset
sentences = treebank.tagged_sents()

[nltk_data] Downloading package treebank to
[nltk_data]     /u/amo-d1/grad/mha361/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
Some weights of the model checkpoint at QCRI/bert-base-multilingual-cased-pos-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [47]:
from torch.utils import data
class PosDataset(data.Dataset):
    def __init__(self, tagged_sents,tokenizer,tag2idx,idx2tag):
        sents, tags_li = [], [] # list of lists
        self.tokenizer = tokenizer
        self.tag2idx = tag2idx
        self.idx2tag = idx2tag
        for sent in tagged_sents:
            words = [word_pos[0] for word_pos in sent]
            tags = [word_pos[1] for word_pos in sent]
            sents.append(["[CLS]"] + words + ["[SEP]"])
            tags_li.append(["<pad>"] + tags + ["<pad>"])
        self.sents, self.tags_li = sents, tags_li

    def __len__(self):
        return len(self.sents)

    def __getitem__(self, idx):
        words, tags = self.sents[idx], self.tags_li[idx] # words, tags: string list

        # We give credits only to the first piece.
        x, y = [], [] # list of ids
        is_heads = [] # list. 1: the token is the first piece of a word
        for w, t in zip(words, tags):
            tokens = self.tokenizer.tokenize(w) if w not in ("[CLS]", "[SEP]") else [w]
            xx = self.tokenizer.convert_tokens_to_ids(tokens)

            is_head = [1] + [0]*(len(tokens) - 1)

            t = [t] + ["<pad>"] * (len(tokens) - 1)  # <PAD>: no decision
            yy = [self.tag2idx[each] for each in t]  # (T,)

            x.extend(xx)
            is_heads.extend(is_head)
            y.extend(yy)

        assert len(x)==len(y)==len(is_heads), "len(x)={}, len(y)={}, len(is_heads)={}".format(len(x), len(y), len(is_heads))

        # seqlen
        seqlen = len(y)

        # to string
        words = " ".join(words)
        tags = " ".join(tags)
        return words, x, is_heads, tags, y, seqlen

In [48]:
tagged_sents = treebank.tagged_sents()
print(len(tagged_sents))
print(tagged_sents[0])
tags = list(set(word_pos[1] for sent in tagged_sents for word_pos in sent))
tags = ["<pad>"] + tags
tags_str = ','.join(tags)
print(len(tags_str))
print(tags_str)
tag2idx = {tag:idx for idx, tag in enumerate(tags)}
idx2tag = {idx:tag for idx, tag in enumerate(tags)}
dataset = PosDataset(sentences,tokenizer,tag2idx,idx2tag)

3914
[('Pierre', 'NNP'), ('Vinken', 'NNP'), (',', ','), ('61', 'CD'), ('years', 'NNS'), ('old', 'JJ'), (',', ','), ('will', 'MD'), ('join', 'VB'), ('the', 'DT'), ('board', 'NN'), ('as', 'IN'), ('a', 'DT'), ('nonexecutive', 'JJ'), ('director', 'NN'), ('Nov.', 'NNP'), ('29', 'CD'), ('.', '.')]
170
<pad>,RP,JJS,RB,TO,JJR,IN,PDT,SYM,WRB,LS,VBN,-RRB-,VB,VBZ,JJ,,,NN,:,-LRB-,``,NNPS,#,PRP$,NNS,$,EX,WDT,VBD,PRP,VBG,.,UH,POS,'',RBR,MD,CD,CC,VBP,-NONE-,FW,WP$,NNP,WP,RBS,DT


In [69]:
from tqdm import tqdm
true_labels = []
pred_labels = []
model.eval()
with torch.no_grad():
    for words, x, is_heads, tags, y, seqlen in tqdm(dataset):
        
        x = torch.tensor(x).unsqueeze(0)
        outputs = model(x).logits
        predictions = torch.argmax(outputs, dim=2).squeeze().tolist()
        
        

  0%|          | 0/3914 [00:38<?, ?it/s]


KeyboardInterrupt: 

In [65]:
true_labels[0]

['<',
 'p',
 'a',
 '>',
 ' ',
 'N',
 'N',
 'P',
 ' ',
 'N',
 'N',
 'P',
 ' ',
 ',',
 ' ',
 'N',
 'N',
 ' ',
 'J',
 'J']

In [66]:
pred_labels[0]

['.',
 'NNP',
 'NNP',
 ',',
 'CD',
 'NNS',
 'JJ',
 ',',
 'MD',
 'VB',
 'DT',
 'NN',
 'IN',
 'DT',
 'JJ',
 'NN',
 'NNP',
 'CD',
 '.',
 '.']